In [0]:
from pyspark.sql.functions import *

cosmosDBEndpoint = "https://scc23rosatiago.documents.azure.com:443/"
cosmosDBKey = "tKcgbM69CKTWXfw9NFFyi37KnRhWdimQMPYEaqGAoIiv9uf2IoJ4lXeIlQsG6uHFlTrMUBE16lppACDbuqAedw=="
cosmosDBName = "scc23dbrosatiago"

try:
    readConfigBids = {
      "spark.cosmos.accountEndpoint": cosmosDBEndpoint,
      "spark.cosmos.accountKey": cosmosDBKey,
      "spark.cosmos.database": cosmosDBName,
      "spark.cosmos.container": "bids",
      "spark.cosmos.read.customQuery": "SELECT * FROM bids"
    }
    
    readConfigAuctions = {
      "spark.cosmos.accountEndpoint": cosmosDBEndpoint,
      "spark.cosmos.accountKey": cosmosDBKey,
      "spark.cosmos.database": cosmosDBName,
      "spark.cosmos.container": "auctions",
      "spark.cosmos.read.customQuery": "SELECT * FROM auctions"
    }

    bids = spark.read.format("cosmos.oltp").options(**readConfigBids) \
                        .option("spark.cosmos.read.inferSchema.enabled", "true") \
                        .load()
    
    auctions = spark.read.format("cosmos.oltp").options(**readConfigAuctions) \
                        .option("spark.cosmos.read.inferSchema.enabled", "true") \
                        .load()
    
    bids.createOrReplaceTempView("bids")
    auctions.createOrReplaceTempView("auctions")
    
    recentBidsCount = spark.sql("""SELECT bids.auctionId, count(bids.auctionId) as count
                        FROM bids
                        WHERE bids.time <= current_timestamp() AND bids.time >= dateadd(DAY, -1, current_timestamp())
                        GROUP BY bids.auctionId""")
    
    recentBidsCount.createOrReplaceTempView("recentBidsCount")
    
    popularAuctionsIds = spark.sql("""SELECT recentBidsCount.auctionId
                        FROM recentBidsCount
                        WHERE recentBidsCount.count >= 5""")
    
    popularAuctionsIds.createOrReplaceTempView("popularAuctionsIds")
    
    popularAuctions = spark.sql("""SELECT auctions.id, auctions.title, auctions.description, auctions.imageId, auctions.ownerId, auctions.endDate, auctions.minimumPrice,                                                   auctions.auctionStatus, auctions.winnerBid
                                   FROM popularAuctionsIds
                                   JOIN auctions
                                   WHERE popularAuctionsIds.auctionId=auctions.id""")
    
    popularAuctions.createOrReplaceTempView("popularAuctions")
    

    result = spark.sql("""SELECT * FROM popularAuctions""")

    # Write configuration
    writeConfig = {
      "spark.cosmos.accountEndpoint": cosmosDBEndpoint,
      "spark.cosmos.accountKey": cosmosDBKey,
      "spark.cosmos.database": cosmosDBName,
      "spark.cosmos.container": "popularAuctions",
    }

    # Write to Cosmos DB from the result DataFrame
    result.write.format("cosmos.oltp").options(**writeConfig) \
                  .mode("append") \
                  .save()
except Exception as e:
  print(e)
